# Practical Exercise: Fine-tune a Model with GRPO

In this Colab-ready notebook, you'll fine-tune a language model using Group Relative Policy Optimization (GRPO) as described in the DeepSeek R1 and Hugging Face TRL documentation. This exercise is adapted from the original by [@mlabonne](https://huggingface.co/mlabonne).

---

**Outline:**
1. Install dependencies
2. Import libraries
3. Log in to Weights & Biases (optional)
4. Load dataset
5. Load model and LoRA
6. Define reward function
7. Define training arguments
8. Train with GRPO
9. Interpret results
10. Save and publish model
11. Generate text

---

In [ ]:
# 1. Install dependencies
!pip install -qqq datasets==3.2.0 transformers==4.47.1 trl==0.14.0 peft==0.14.0 accelerate==1.2.1 bitsandbytes==0.45.2 wandb==0.19.7 --progress-bar off
!pip install -qqq flash-attn --no-build-isolation --progress-bar off

In [ ]:
# 2. Import libraries
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOConfig, GRPOTrainer

In [ ]:
# 3. Log in to Weights & Biases (optional)
import wandb
wandb.login()  # You can skip this cell if you don't want to log runs

In [ ]:
# 4. Load dataset
dataset = load_dataset("mlabonne/smoltldr")
print(dataset)

In [ ]:
# 5. Load model and LoRA
model_id = "HuggingFaceTB/SmolLM-135M-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype="auto",
    device_map="auto",
    attn_implementation="flash_attention_2",
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Load LoRA
lora_config = LoraConfig(
    task_type="CAUSAL_LM",
    r=16,
    lora_alpha=32,
    target_modules="all-linear",
)
model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

In [ ]:
# 6. Define the reward function
ideal_length = 50

def reward_len(completions, **kwargs):
    return [-abs(ideal_length - len(completion)) for completion in completions]

In [ ]:
# 7. Define the training arguments
training_args = GRPOConfig(
    output_dir="GRPO",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=2,
    max_prompt_length=512,
    max_completion_length=96,
    num_generations=8,
    optim="adamw_8bit",
    num_train_epochs=1,
    bf16=True,
    report_to=["wandb"],
    remove_unused_columns=False,
    logging_steps=1,
)

In [ ]:
# 8. Train with GRPO
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[reward_len],
    args=training_args,
    train_dataset=dataset["train"],
)

import wandb
wandb.init(project="GRPO")
trainer.train()

# 9. Interpret training results

`GRPOTrainer` logs the reward from your reward function, the loss, and a range of other metrics.

- The reward should move closer to 0 as the model learns to generate text of the correct length.
- The loss may increase as the model diverges from its initial policy, which is expected in GRPO.

You can view the training logs in your Weights & Biases dashboard if you logged in.

In [ ]:
# 10. Save and publish the model
# (Optional) Push the model to the Hugging Face Hub
# merged_model = trainer.model.merge_and_unload()
# merged_model.push_to_hub(
#     "SmolGRPO-135M", private=False, tags=["GRPO", "Reasoning-Course"]
# )

In [ ]:
# 11. Generate text with the fine-tuned model
from transformers import pipeline

prompt = """
# A long document about the Cat

The cat (Felis catus), also referred to as the domestic cat or house cat, is a small 
domesticated carnivorous mammal. It is the only domesticated species of the family Felidae.
Advances in archaeology and genetics have shown that the domestication of the cat occurred
in the Near East around 7500 BC. It is commonly kept as a pet and farm cat, but also ranges
freely as a feral cat avoiding human contact. It is valued by humans for companionship and
its ability to kill vermin. Its retractable claws are adapted to killing small prey species
such as mice and rats. It has a strong, flexible body, quick reflexes, and sharp teeth,
and its night vision and sense of smell are well developed. It is a social species,
but a solitary hunter and a crepuscular predator. Cat communication includes
vocalizations—including meowing, purring, trilling, hissing, growling, and grunting—as
well as body language. It can hear sounds too faint or too high in frequency for human ears,
such as those made by small mammals. It secretes and perceives pheromones.
"""

messages = [
    {"role": "user", "content": prompt},
]

generator = pipeline("text-generation", model="SmolGRPO-135M")
# Or use the model and tokenizer you defined earlier:
# generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

generate_kwargs = {
    "max_new_tokens": 256,
    "do_sample": True,
    "temperature": 0.5,
    "min_p": 0.1,
}

generated_text = generator(messages, generate_kwargs=generate_kwargs)
print(generated_text)